In [ ]:
## Please verify that you have installed the python libraries from requirements.txt

## You can install them with ```pip install -r requirements.txt```
## Other versions may work but they are not tested.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# set desired gpu device
import os; os.environ['CUDA_VISIBLE_DEVICES'] = '0';
import gc
import re
import json
import time
import glob
import random
import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter

import numpy as np
import pandas as pd

from sklearn import model_selection
from scipy.special import expit

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers.optimization import AdamW
from transformers import RobertaConfig

from utils import processing_mimic3, get_score, RobertaForICD9, RobertaForICD9_wo_positional

SEED = 79
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# change to the correct path to MIMIC3 data files, e.g. '/ayb/vol3/datasets/MIMIC3/'
MIMIC_PATH = '?'
# this is the name of preprocessed file
MIMIC_data_file = 'patients_mimic3_full.json'

In [ ]:
## prepare data file for all patients from raw MIMIC3 data
file_adm = os.path.join(MIMIC_PATH, 'ADMISSIONS.csv')
file_dxx = os.path.join(MIMIC_PATH, 'DIAGNOSES_ICD.csv')
file_txx = os.path.join(MIMIC_PATH, 'PROCEDURES_ICD.csv')
file_drug = os.path.join(MIMIC_PATH, 'PRESCRIPTIONS.csv')
file_drg = os.path.join(MIMIC_PATH, 'DRGCODES.csv')

processing_mimic3(file_adm, file_dxx, file_txx, file_drug, file_drg, MIMIC_data_file)

In [ ]:
## Uncomment if you would like to download the file AppendixCMultiDX.txt from the original source
# !wget -c --tries=0 --read-timeout=10 "https://www.hcup-us.ahrq.gov/toolssoftware/ccs/AppendixCMultiDX.txt" -O AppendixCMultiDX.txt

In [ ]:
# build map 'icd9 code' -> 'second level category' from ICD9 hierarchy
with open('AppendixCMultiDX.txt') as f:
    lines = f.readlines()
    
categ = ''
code_lines = []
cat2line = {}
for line in lines:
    is_name_line = line[0].isdigit()
    if is_name_line: # start of a new categ
        cat2line[categ] = code_lines
        
        categ = line
        
        parts = line.split(' ')
        index = parts[0]
        categ = index
        
        code_lines = []
    else:
        code_lines.append(line)
cat2line[categ] = code_lines

fcat2codes = {} #full category to code
for cat in cat2line:
    if len(cat)>0:
        codes = set()
        for line in cat2line[cat]:
            line = re.sub(' +', ' ', line).strip()
            if len(line)>0:
                subset = line.split(' ')
                codes.update( subset )
        if len(codes)>0:
            fcat2codes[cat] = codes
            
unique_categs = set() # unique second level categories
for i in fcat2codes.keys():
    parts = i.split('.')
    if len(parts)>0:
        cat = '.'.join(parts[:2])
        unique_categs.add(cat)
        
cat2codes = {}
for i, cat in enumerate(sorted(list(unique_categs))):
    codes = set()
    for key in fcat2codes:
        trimed_key = '.'.join(key.split('.')[:2])
        if trimed_key==cat:
            subset = fcat2codes[key]
            codes.update( subset )
    cat2codes[i] = sorted(list(codes))
    
code2categ = {}
for cat in cat2codes:
    codes = cat2codes[cat]
    for code in codes:
        code2categ[code] = cat
print (len(code2categ))

num_categs = len(set(code2categ.values()))

print (num_categs)

In [ ]:
## len(code2categ) == 15072
## num_categs == 136

In [ ]:
# set params to filter out histories and diagnoses
min_visits_number = 2
infrequent_diagnoses = 5

In [ ]:
with open(MIMIC_data_file) as f:
    jdata = json.load(f)
print (len(jdata))

In [ ]:
diagnoses = []
for rec in jdata:
    for v in rec['visits']:
        diagnoses.extend( v['DXs'] )
diagnoses = dict(Counter(diagnoses))

left_diagnoses = set([code for code in diagnoses if diagnoses[code] >= infrequent_diagnoses])

In [ ]:
selected_patients = []

for rec in jdata:
    if len(rec['visits']) < min_visits_number:
        continue
    
    ok_visit = 0
    for v in rec['visits']:
        current_DXs = set( v['DXs'] )
        leftover = current_DXs.intersection(left_diagnoses)
        if len(leftover)>0:
            ok_visit += 1
    
    if ok_visit >= min_visits_number:
        selected_patients.append(rec)
selected_patients = np.array(selected_patients)
print (len(selected_patients))

In [ ]:
mean_visits = np.mean([len(rec['visits']) for rec in selected_patients])
print (mean_visits)

In [ ]:
## len(selected_patients) == 7496
## mean_visits == 2.6562166488794023

In [ ]:
df = pd.read_csv( os.path.join(MIMIC_PATH, 'PATIENTS.csv') )

In [ ]:
pid2gender = {str(pid):1 if g=='M' else 0 for pid, g in zip(df.SUBJECT_ID, df.GENDER)}
pid2dob = {str(pid):int(time.mktime(datetime.datetime.strptime(g.split(' ')[0], "%Y-%m-%d").timetuple())) for pid, g in zip(df.SUBJECT_ID, df.DOB)}

In [ ]:
# recs is array of tuples (gender, age, history_codes, the_last_visit_codes)
recs = []
for rec in selected_patients:
    pid = rec['pid']
    gender = pid2gender[pid]
    
    #sort visits by time
    timestamps = []
    for v in rec['visits']:
        admsn_dt = v['admsn_dt']
        ts = int(time.mktime(datetime.datetime.strptime(admsn_dt, "%Y%m%d").timetuple()))
        timestamps.append(ts)
    order = np.argsort(timestamps)
    
    rec['visits'] = list( np.array(rec['visits'])[order] )
    
    history = [] #collect history up to last visit
    for v in rec['visits'][:-1]:
        for diag9 in v['DXs']:
            history.append( diag9 )
    
    # compute the patient age at pre last visit
    ts = int(time.mktime(datetime.datetime.strptime(v['admsn_dt'], "%Y%m%d").timetuple()))
    age = relativedelta(datetime.datetime.fromtimestamp(ts), datetime.datetime.fromtimestamp(pid2dob[pid])).years
    age = min(89, age) # to overcome obscure in MIMIC
    
    target = []
    v = rec['visits'][-1]
    for diag9 in v['DXs']:
        target.append( diag9 )
    
    recs.append( (gender, age, history, target) )
recs = np.array(recs, dtype=object)
print (recs.shape)

In [ ]:
recs[1]

In [ ]:
#number of auxilary tokens is 4: 0(start), 1(pad), 2(end), 3(mask)
#plus couple tokens for gender: 4th index for women, 5th index for men
#plus token indicies for each reasonable age: 0-99
code_shift = 4+\
2+\
100

In [ ]:
all_codes = []
for rec in recs:
    all_codes.extend(rec[2])
    all_codes.extend(rec[3])
all_codes = set(all_codes)
print (len(all_codes))

In [ ]:
code2index = {}
counter = 0
for c in sorted(list(all_codes)):
    # ICD token indices starts after 99+{0/1}+4
    code2index[c] = code_shift + counter
    counter+=1
print (len(code2index))

index2code = {code2index[c]:c for c in code2index}

In [ ]:
num_cases = 128 # consider the patient history no more than 'num_cases'==128 elemens

# rec vectorizer funcition
def vectorize(subset):
    vectors, targets = [], []
    for rec in subset:
        #sex / age components
        pref = [
            0, #start sequence token
            # 4 is the number of auxilary tokens
            4 + rec[0], # 4th for women, 5th token for men
            6 + rec[1] # token index for each posible age
        ] # ang and sex prefix

        history = np.ones( num_cases+1 )

        # convert ICD codes to sequence of token indices
        postf = []
        for c in rec[2][:num_cases]:
            if c in code2index:
                postf.append( code2index[c] )
        postf.append(2) # end sequence token

        history[:len(postf)] = postf

        history = np.concatenate([pref, history])

        vectors.append(history)
        
        t_out = np.zeros(num_categs)
        for c in rec[3]:
            idx = code2categ[c]
            t_out[idx] = 1
        targets.append(t_out)
    targets = np.array(targets)
        
    vectors = np.array(vectors)
    return vectors, targets

In [ ]:
batch_size = 256
epochs = 19
lr = 9e-5
max_grad_norm = 1.0

scores = []
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=SEED)
for fold_num, (train_index, cv_index) in enumerate(kf.split(recs), start=1):
    X_train, y_train = vectorize( recs[train_index] )    
    X_cv, y_cv = vectorize( recs[cv_index] )
    
    vocab_size = np.max( list(code2index.values()) )
    hidden_dim = 256
    layer_and_heads = 4

    config = RobertaConfig(
        bos_token_id=0,
        pad_token_id=1,
        eos_token_id=2,

        initializer_range=.02,

        hidden_size=hidden_dim,
        intermediate_size=hidden_dim*2,

        max_position_embeddings=X_train.shape[1]+2,
        num_attention_heads=layer_and_heads,
        num_hidden_layers=layer_and_heads,
        type_vocab_size=4,

        vocab_size=vocab_size
    )
    
    config.num_labels = num_categs
    #print (config)
    #model = RobertaForICD9(config=config)
    model = RobertaForICD9_wo_positional(config=config)
    #model.num_parameters()
    model = model.to(device)
    
    ##train_dataloader
    input_ids_train = torch.tensor( X_train.astype(int) )
    attention_masks_train = torch.tensor( (X_train!=1).astype(int) )
    labels_train = torch.tensor( y_train )
    train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data,
        sampler=train_sampler,
        batch_size=batch_size,
        num_workers=4,
        worker_init_fn=seed_worker
    )

    ##prediction_dataloader
    input_ids_test = torch.tensor( X_cv.astype(int) )
    attention_masks_test = torch.tensor( (X_cv!=1).astype(int) )
    labels_test = torch.tensor( y_cv )
    prediction_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(
        prediction_data,
        sampler=prediction_sampler,
        batch_size=batch_size*2,
        num_workers=4,
        worker_init_fn=seed_worker
    )

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    train_loss = []
    for _ in range(epochs):
        model.train(); torch.cuda.empty_cache()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0

        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()

            # Forward pass
            outputs = model( b_input_ids, attention_mask=b_input_mask, labels=b_labels )
            loss, logits = outputs[:2]
            train_loss.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()

            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
        tr_loss_value = tr_loss/nb_tr_steps

        ### val
        model.eval()
        predictions = []
        tr_loss, nb_tr_steps = 0, 0
        for step, batch in enumerate(prediction_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up prediction
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                outputs = model( b_input_ids, attention_mask=b_input_mask, labels=b_labels )
                loss, logits = outputs[:2]
                tr_loss += loss.item()
                nb_tr_steps += 1
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            # Store predictions and true labels
            predictions.append(logits)
        predictions = expit(np.vstack(predictions))
        val_loss_value = tr_loss/nb_tr_steps
        
        scheduler.step(val_loss_value)
        
        score5 = get_score(y_cv, predictions)
        
        print ( 'P@5:{}\tEPOCH {} train loss: {} val loss: {}'.format(score5, _, tr_loss_value, val_loss_value) )
    
    fold_scores = [get_score(y_cv, predictions, k=k) for k in [5, 10, 20, 30]]
    scores.append(fold_scores)
    print ('\tFold {}: {}'.format(fold_num, fold_scores))

scores = np.array(scores)
precision_k_means = np.round(
    np.mean(scores, axis=0),
    decimals=4
)

precision_k_stds = np.round(
    np.std(scores, axis=0),
    decimals=4
)

print ('\nPrecision@k')
print ('means', precision_k_means)
print ('stds', precision_k_stds)

In [ ]:
# Precision@k
# means [0.6677 0.619  0.7258 0.8217]
# stds [0.0084 0.0099 0.0083 0.0045]